# Coursera IBM capstone project -  The Battle of Neighborhoods

<p> project title: </p>

<h2>Finding a proper location for opening a new restaurant in the fututre that is suitable during the ‘covid - 19’ pandemic in Riyadh, Saudi Arabia using Data science and analytical tools</h2>

<p> Author: Raihan Insan Habibi</p>

<b>1. Introduction / business problems</b>

On 4th May 2020, Saudi Arabia was struck by a deadly virus called the coronavirus also known by its scientific name 'covid-19'. and on the 9th of September 2020, the recovery rate in Saudi Arabia has increased dramatically because of the new-normal systems. social distancing has changed many methods for many businesses, especially for restaurants. Since things are slowly going back to normal, people who want to open up a restaurant in the future will have to have to find a suitable location that will be effective for business, even during the 'covid-19' pandemic. Finding a proper location requires various factors that data scientists need to consider and process, as well as all of the access to data information around the neighbourhood. In this report, I will be describing a simple analysis of data from Riyadh, Saudi Arabia city neighbourhood to find a decent location to open up a new restaurant.

<b>2. Data</b>

(all data resources will be here)

In [104]:
#!pip install geopy
#!pip install folium
#!pip install bs4

You should consider upgrading via the 'c:\users\mathl\anaconda3\python.exe -m pip install --upgrade pip' command.


In [108]:
import numpy as np
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as Soup
print("libaries imported")

libaries imported


In [98]:
CLIENT_ID = 'P2X2VEDCLZQPBJVLBKFLTJIOJIMQENGZHXXCOLDAAAP1ATVA' 
CLIENT_SECRET = 'TX5XWBRYCABQ0DNWW110SNY12PZE242WYLDFKXFKREV3R3KX' 
VERSION = '20180604'
LIMIT = 30

address = '11564, Saudi Arabia, Riyadh'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
coords = (latitude, longitude)
print(coords)
Riyadh_map = folium.Map(coords)
search_query = 'restaurant'
radius = 2000
LIMIT = 100

(24.6161141, 46.7056208)


In [99]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
url

'https://api.foursquare.com/v2/venues/search?client_id=P2X2VEDCLZQPBJVLBKFLTJIOJIMQENGZHXXCOLDAAAP1ATVA&client_secret=TX5XWBRYCABQ0DNWW110SNY12PZE242WYLDFKXFKREV3R3KX&ll=24.6161141,46.7056208&v=20180604&query=restaurant&radius=2000&limit=100'

In [100]:
venues = results['response']['venues']
df = json_normalize(venues)
df.head()

<ipython-input-100-8c85c09aef9d>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.city,location.state,location.crossStreet,location.postalCode
0,5244c1fc11d22a457711e2da,Forouj Al carawan Restaurant,"[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",v-1614295244,False,24.603433,46.693419,"[{'label': 'display', 'lat': 24.60343279032182...",1875,SA,المملكة العربية السعودية,[المملكة العربية السعودية],NaN,NaN,NaN,NaN,NaN
1,5229b01111d27c467fc7f6d7,Redan Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1614295244,False,24.637627,46.701290,"[{'label': 'display', 'lat': 24.63762676804035...",2434,SA,المملكة العربية السعودية,[المملكة العربية السعودية],NaN,NaN,NaN,NaN,NaN
2,56556136498e006b1b195e46,Alshera3 Restaurant (مطاعم الشراع),"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1614295244,False,24.601332,46.697614,"[{'label': 'display', 'lat': 24.60133236677082...",1834,SA,المملكة العربية السعودية,"[شارع السويدي العام, الرياض, المملكة العربية ا...",شارع السويدي العام,الرياض,منطقة الرياض‎,NaN,NaN
3,55f2d0c9498ecdec7378c2bd,Tekkady Restaurant (مطعم تيكادي),"[{'id': '54135bf5e4b08f3d2429dfdf', 'name': 'I...",v-1614295244,False,24.638121,46.702265,"[{'label': 'display', 'lat': 24.638121, 'lng':...",2473,SA,المملكة العربية السعودية,"[King Fahd Rd, الرياض, المملكة العربية السعودية]",King Fahd Rd,الرياض,منطقة الرياض‎,NaN,NaN
4,55f5be43498e1449d84fd3e7,Suha's Multi Cuisine Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614295244,False,24.608172,46.710604,"[{'label': 'display', 'lat': 24.60817223004661...",1017,SA,المملكة العربية السعودية,[المملكة العربية السعودية],NaN,NaN,NaN,NaN,NaN


In [101]:
#cleaning data
print(df.columns.tolist())
df = df.drop(["id", "categories","location.country","location.state",'location.postalCode', "referralId", "hasPerk",'location.labeledLatLngs', 'location.cc','location.formattedAddress' ], axis = 1)
df = df.rename(columns = {"name":"Restaurant",
                          "location.lat":"latitude", 
                          'location.lng':"longitude",
                          'location.distance':"distance",
                          'location.address':"address", 
                          "location.city":"city",
                          'location.crossStreet':"street name", 
                         })
new_distance = []
df["distance"] = df["distance"].astype(str)
df["distance"] = df["distance"].tolist()
for x in df["distance"]:
    y = x + "km"
    new_distance.append(y)

df["distance"] = new_distance


['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.lat', 'location.lng', 'location.labeledLatLngs', 'location.distance', 'location.cc', 'location.country', 'location.formattedAddress', 'location.address', 'location.city', 'location.state', 'location.crossStreet', 'location.postalCode']


In [102]:
df

,Restaurant,latitude,longitude,distance,address,city,street name
0,Forouj Al carawan Restaurant,24.603433,46.693419,1875km,NaN,NaN,NaN
1,Redan Restaurant,24.637627,46.701290,2434km,NaN,NaN,NaN
2,Alshera3 Restaurant (مطاعم الشراع),24.601332,46.697614,1834km,شارع السويدي العام,الرياض,NaN
3,Tekkady Restaurant (مطعم تيكادي),24.638121,46.702265,2473km,King Fahd Rd,الرياض,NaN
4,Suha's Multi Cuisine Restaurant,24.608172,46.710604,1017km,NaN,NaN,NaN
5,Said Turkish Restaurant مطعم سعيد التركي,24.625771,46.694054,1589km,Assir st. شارع عسير,الرياض,Madinah Rd طريق المدينة Ojaila Street
6,Peshawari Restaurant,24.624977,46.697323,1295km,"7359 Al Umraniyah, Ash Shimaisi, Riyadh 12745 ...",الرياض,Al Jandal Street
7,Gad (جاد),24.637952,46.706148,2431km,Fouta District,الرياض,Khazzan St.
8,Indian Summer Restaurant مطعم الصيف الهندي,24.616782,46.718141,1269km,3aeshah bent abi baker,NaN,NaN
9,AlDafwa Restaurant,24.629652,46.709054,1546km,Malaz Area,الرياض,Al Ahsa St.


In [122]:
Riyadh_map = folium.Map(coords, zoom_start = 14)
#get coords from table
latitude = df["latitude"].tolist()
longitude = df["longitude"].tolist()
restaurant = df["Restaurant"].tolist()

for n in range(len(latitude)):
    coords = latitude[n],longitude[n]
    folium.Marker([coords[0], coords[1]], popup = restaurant[n]).add_to(Riyadh_map)
    
Riyadh_map


In [111]:
my_url = "https://www.google.com/maps/search/saudi+arabia,+riyadh,+residents/@24.7984497,46.6521377,10z"
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = Soup(page_html, "html.parser")